In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra\
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brighton-a-memorable-city/training.csv
/kaggle/input/brighton-a-memorable-city/annotation_confidence.csv
/kaggle/input/brighton-a-memorable-city/testing.csv
/kaggle/input/brighton-a-memorable-city/test_proportions.csv
/kaggle/input/brighton-a-memorable-city/sample_valid_submission.csv
/kaggle/input/brighton-a-memorable-city/additional_training.csv


In [68]:
#imports needed 
from sklearn.impute import KNNImputer
from sklearn.neural_network import MLPClassifier

In [69]:
#loading and storing the csv files into variables
trainingData = pd.read_csv('/kaggle/input/brighton-a-memorable-city/training.csv')
additionalTrainingData = pd.read_csv('/kaggle/input/brighton-a-memorable-city/additional_training.csv')
testingData = pd.read_csv('/kaggle/input/brighton-a-memorable-city/testing.csv')

In [70]:
#storing prediction values from training and additional training data into variables
trainingPredictions = trainingData['prediction']
additionalTrainingPredictions = additionalTrainingData['prediction']
#storing ID values from thestng data into variable
identificationNumberTesting = testingData['ID']

In [71]:
#storing the data frae from the different data files into variables
trainingDataFrame = pd.DataFrame(trainingData)
additionalTrainingDataFrame = pd.DataFrame(additionalTrainingData)
testingDataFrame = pd.DataFrame(testingData)

In [72]:
#saving into variables features within the different data sets
#I have used the iloc in order to do this as it allows for purely integer-location indexing for selecting a position within panda data frame
trainingFeatures = trainingDataFrame.iloc[:247, 1:4609]
additionalTrainingFeatures = additionalTrainingDataFrame.iloc[:2219, 1:4609]
testingFeatures = testingDataFrame.iloc[:11874, 1:4609]

In [73]:
#declaring the knnimputer and adjusting the hyper-parameters for it
missingValuesImputer = KNNImputer(n_neighbors = 3, missing_values = np.nan)
#using the declared imputer in order to enter the missing features values into the additional training data set
additionalTrainingFeatures = missingValuesImputer.fit_transform(additionalTrainingFeatures)
#storing the values in the data frame format
additionalTrainingFeatures = pd.DataFrame(additionalTrainingFeatures)

In [74]:
#merging the training features values with the additional training features values
#I have used the concatenate function as the values have been stored within a numpy array
allTrainingFeatures = np.concatenate([trainingFeatures, additionalTrainingFeatures])
#merging the training prediction values with the additional training prediction values
#I have used the cabcat function as the values have been stored panda data set
allTrainingPredictions = pd.concat([trainingPredictions, additionalTrainingPredictions])

In [104]:
#declaring and setting the hyper parameters for the MLPClassifier
classifier = MLPClassifier(solver = 'lbfgs', alpha = 2e-2, random_state = 8)
#using the merged training features and training predictions in order to train the model
classifier.fit(allTrainingFeatures, allTrainingPredictions)

MLPClassifier(activation='relu', alpha=0.02, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=8, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [105]:
#using the classifier to predict values for the testing features and storing it as a variable
predictions = classifier.predict(testingFeatures)

In [106]:
#intializing a variable that will store a number representing the percentage of zeros within the predictions variable
precentageOfNotMemrable = 0
#intializing a variable that will store a number representing the percentage of ones within the predictions variable
precentageOfMemrable = 0
#getting the number that will represent 1/4 of the values from the predictions as kaggle uses only 1/4 of data to provide the score
quoterOfPredictions = int(len(predictions)/4)

#for loop that will go through the first quoter of predictions 
for i in range(quoterOfPredictions):
    #if the prediction is one it will increment the percentage of memrable photos
    if(predictions[i] == 1):
        precentageOfMemrable += 1
    #if the prediction is zero it will increment the percentage of non memrable photos
    else:
        precentageOfNotMemrable += 1

#calculation that will transform the number into precentage
precentageOfMemrable = precentageOfMemrable / quoterOfPredictions * 100
precentageOfNotMemrable = precentageOfNotMemrable / quoterOfPredictions * 100
#print out statments that will be used to display the precentages with appropriate message
print("Precentage of memrable photos: " + str(round(precentageOfMemrable)) + "%")
print("Precentage of not memrable photos: " + str(round(precentageOfNotMemrable)) + "%")

Precentage of memrable photos: 55%
Precentage of not memrable photos: 45%


In [100]:
#this will store the predictions in the csv file that can be submitted to the kaggle competion
finalDataFrame = pd.DataFrame(zip(identificationNumberTesting, predictions), columns = ["ID", "Prediction"])
finalDataFrame.to_csv('./predictions.csv', index = False, header = True)